In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from scipy.stats import pearsonr

In [39]:
test_df = pd.read_csv('test.csv')
pred_cols = ['distil_pred', 'xlm_r_pred', 'xlm_t_pred', 'debert_pred']
test_df.columns

Index(['text', 'label', 'language', 'distil_pred', 'multi_pred', 'xlm_r_pred',
       'xlm_t_pred', 'debert_pred'],
      dtype='object')

In [37]:
train_df = pd.read_csv('train.csv')
# train_df = train2
train_df.columns

Index(['text', 'label', 'language', 'xlm_t_pred', 'xlm_r_pred', 'distil_pred',
       'debert_pred'],
      dtype='object')

In [42]:
train_scaler = StandardScaler()
#train_df[pred_cols] = train_scaler.fit_transform(train_df[pred_cols])
X_train = train_scaler.fit_transform(train_df[pred_cols])
X_test = train_scaler.transform(test_df[pred_cols])

label_scaler = StandardScaler()
y_train = label_scaler.fit_transform(train_df['label'].values.reshape(-1, 1))
y_test = label_scaler.transform(test_df['label'].values.reshape(-1, 1))
y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [48]:
stack_model = LinearRegression()
stack_model.fit(X_train, y_train)

y_pred = stack_model.predict(X_test)
test_df['stack_pred'] = label_scaler.inverse_transform(y_pred.reshape(-1, 1)).squeeze()

In [49]:
test_df['avg_pred'] = test_df[pred_cols].mean(axis=1)

In [51]:
cor_list1 = []
cor_list2 = []
language_list = test_df['language'].unique()
for lang in language_list:
    lang_df = test_df[test_df['language'] == lang]
    corr_stack, _ = pearsonr(lang_df['label'], lang_df['stack_pred'])
    cor_list1.append(corr_stack)
    corr_avg, _ = pearsonr(lang_df['label'], lang_df['avg_pred'])
    cor_list2.append(corr_avg)

cor_df = pd.DataFrame({'language': language_list, 'stack_corr': cor_list1, 'avg_corr': cor_list2})
total_corr_stack, _ = pearsonr(test_df['label'], test_df['stack_pred'])
total_corr_avg, _ = pearsonr(test_df['label'], test_df['avg_pred'])
print('Total correlation with stack:', total_corr_stack, total_corr_avg)
cor_df

Total correlation with stack: 0.5983060113475012 0.5970805802324285


,language,stack_corr,avg_corr
0,Korean,0.378092,0.373683
1,French,0.693277,0.699726
2,English,0.700931,0.705324
3,Italian,0.722706,0.724223
4,Chinese,0.733498,0.737349
5,Spanish,0.720840,0.721951
6,Hindi,0.214868,0.226872
7,Arabic,0.595549,0.588777
8,Portuguese,0.673658,0.674833
9,Dutch,0.602379,0.606787
